<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark10_movement_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')


Mounted at /content.gdrive


In [101]:
import json
import numpy as np
import cv2

with open('/content.gdrive/MyDrive/Test2/exp53/predictions.json', "r") as fp:
    data = json.load(fp)

print(type(data["x_center"]))
print(data["x_center"])
# convert lists to numpy arrays
x_center = np.array(data["x_center"])
y_center = np.array(data["y_center"])
x1 = np.array(data["x1"])
x2 = np.array(data["x2"])
y1 = np.array(data["y1"])
y2 = np.array(data["y2"])
det = np.array(data["det"])
width = 1920
hight = 1080


<class 'list'>
[1276.4999771118164, 1277.9999542236328, 347.49999046325684, 1145.0000381469727, 416.49999618530273, 956.0000038146973, 720.9999847412109, 1232.4999618530273, 346.5000057220459, 416.49999618530273, 1276.9999694824219, 1144.5000457763672, 956.9999885559082, 720.4999923706055, 1456.4999771118164, 416.49999618530273, 1276.4999771118164, 1145.5000305175781, 956.0000038146973, 347.49999046325684, 720.4999923706055, 1155.4999923706055, 1275.999984741211, 956.4999961853027, 416.00000381469727, 1145.5000305175781, 348.0000114440918, 720.0, 350.50000190734863, 416.49999618530273, 1275.4999923706055, 956.0000038146973, 1145.0000381469727, 1115.0000381469727, 720.4999923706055, 1095.999984741211, 1276.9999694824219, 350.50000190734863, 416.9999885559082, 956.4999961853027, 1145.0000381469727, 719.5000076293945, 1064.000015258789, 956.9999885559082, 1276.4999771118164, 350.50000190734863, 1145.5000305175781, 416.49999618530273, 720.0, 837.4999809265137, 1277.4999618530273, 956.99998

In [42]:
print(sum(det[:1]))
print(det)
print(sum(det[:0+1]))
print(sum(det[:0])+0)
print(math.sqrt((x_center[sum(det[:0+1])]-x_center[sum(det[:0])+0])**2 + (y_center[sum(det[:0+1])]-y_center[sum(det[:0])+0])**2))

list_soll = [1,0,'X',0,5,0,0,7,6,2,4,6,2,2,5,0,1,4,1,0,2]


7
[7 7 7 7 7 7 7 8 7 7 8 8 8 6 9 6 8 8 8 7 9]
7
0
115.69361771412606


In [102]:
#detect movement
import math
from collections import OrderedDict

athlet_det_index = []
y_centers = []

for y in range(21):
    distance = []
    distance_index = []
    valid_dist_index = []
    y_center_per_det = []
    if y < 20:
        for x in range(det[y+1]):
            y_center_per_det.append(y_center[sum(det[:y])+x])
            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        
              
    else:
        for x in range(det[y-1]):

            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        #print(distance)
        #print(distance_index)

    print(y_center_per_det)
    y_centers.append(y_center_per_det)
    for i, x in enumerate(distance):
        if x > 15 and x < 50:
            valid_dist_index.append(distance_index[i])
    
    print(len(y_centers))
    print(len(y_center_per_det))
    print(len(athlet_det_index))
    print("bevor", valid_dist_index)
    #print("athlet", athlet_det_index)
    if len(valid_dist_index) == 0:
        athlet_det_index.append('X')
        """
        #athlet_det_index.append('X')
        distance = []
        distance_index = []
        valid_dist_index = []
        y_center_per_det = []        
        for x in range(det[y-1]):

            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        #y_centers.append(y_center_per_det)
        for i, x in enumerate(distance):
            if x > 15 and x < 50:
                valid_dist_index.append(distance_index[i])
        print("BPBP", valid_dist_index)
        """
        

    if len(valid_dist_index) > 1:
        valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index)) 
    #print(len(valid_dist_index))
    if len(valid_dist_index) == 1:
        athlet_det_index.append(valid_dist_index[0])
      



    if len(valid_dist_index) > 1:
        dist_to_previous_y_center = []
        #print("for", valid_dist_index)
        for i, x in enumerate(valid_dist_index):
            print("x",x)
            print("y",y)
            print(len(y_centers))
            print(len(y_center_per_det))
            print(len(athlet_det_index))
            #print("abs", abs(y_center_per_det[x]-y_centers[y-1][athlet_det_index[y-1]]))
            dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_centers[y-1][athlet_det_index[y-1]]))

        #print("dist", dist_to_previous_y_center)

        closest = min(dist_to_previous_y_center)
        min_index = dist_to_previous_y_center.index(closest)
        athlet_det_index.append(valid_dist_index[min_index])
        #print("BP",valid_dist_index[min_index])




    






print(list_soll)
print(athlet_det_index)







[540.9999704360962, 646.0000205039978, 1015.4999756813049, 553.4999871253967, 602.5000190734863, 565.5000185966492, 576.5000152587891]
1
7
0
bevor [1]
[648.0000257492065, 1014.9999904632568, 601.9999694824219, 542.0000052452087, 553.9999723434448, 564.9999690055847, 577.4999856948853]
2
7
1
bevor []
[547.4999713897705, 602.5000190734863, 543.5000252723694, 553.4999871253967, 564.9999690055847, 1012.5, 577.0000004768372]
3
7
2
bevor []
[663.500018119812, 542.9999756813049, 565.5000185966492, 602.5000190734863, 553.4999871253967, 1012.9999852180481, 577.9999709129333]
4
7
3
bevor [0]
[1012.9999852180481, 603.0000042915344, 543.5000252723694, 565.5000185966492, 553.4999871253967, 659.0000224113464, 576.000030040741]
5
7
4
bevor [5]
[671.4999747276306, 542.9999756813049, 1012.0000147819519, 602.5000190734863, 565.5000185966492, 553.4999871253967, 577.9999709129333]
6
7
5
bevor [0]
[677.0000052452087, 565.5000185966492, 543.5000252723694, 1012.0000147819519, 553.9999723434448, 602.500019073

In [112]:
for i,x in enumerate(athlet_det_index):
    if x != 'X':
        x_1 = x1[sum(det[:i])+x]
        x_2 = x2[sum(det[:i])+x]
        y_1 = y1[sum(det[:i])+x]
        y_2 = y2[sum(det[:i])+x]

        path = '/content.gdrive/MyDrive/Test2/' + str(i+1) + '.jpg'
        print(path)
        im = cv2.imread(path)
        print(x_1, x_2, y_1, y_2)
        crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
        #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
        save_path = '/content.gdrive/MyDrive/Test2/test/' + str(i+1) + '.jpg'
        cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer


/content.gdrive/MyDrive/Test2/1.jpg
1254.0 1302.0 593.0 699.0
/content.gdrive/MyDrive/Test2/4.jpg
1124.0 1187.0 612.0 715.0
/content.gdrive/MyDrive/Test2/5.jpg
1125.0 1165.0 529.0 578.0
/content.gdrive/MyDrive/Test2/6.jpg
699.0 742.0 546.0 606.0
/content.gdrive/MyDrive/Test2/7.jpg
1125.0 1165.0 529.0 578.0
/content.gdrive/MyDrive/Test2/8.jpg
309.0 395.0 946.0 1078.0
/content.gdrive/MyDrive/Test2/9.jpg
1126.0 1165.0 529.0 578.0
/content.gdrive/MyDrive/Test2/10.jpg
698.0 741.0 547.0 608.0
/content.gdrive/MyDrive/Test2/11.jpg
699.0 741.0 546.0 607.0
/content.gdrive/MyDrive/Test2/12.jpg
309.0 395.0 943.0 1080.0
/content.gdrive/MyDrive/Test2/13.jpg
399.0 436.0 579.0 626.0
/content.gdrive/MyDrive/Test2/14.jpg
1127.0 1166.0 528.0 578.0
/content.gdrive/MyDrive/Test2/15.jpg
751.0 793.0 662.0 701.0
/content.gdrive/MyDrive/Test2/16.jpg
939.0 977.0 547.0 586.0
/content.gdrive/MyDrive/Test2/17.jpg
938.0 977.0 546.0 586.0
/content.gdrive/MyDrive/Test2/18.jpg
698.0 742.0 548.0 606.0
/content.gdrive/M